In [43]:
import pandas as pd

# بارگذاری فایل
df = pd.read_csv('gen_brg_vib.csv')

# لیست AssetIDهای مورد نظر
asset_ids = [9362, 9363, 9364, 9365, 9366, 9367, 9371, 9372]

# لیست ستون‌های مورد نظر
columns = ['Value', 'RecordTime', 'RecordDate', 'DateTime', 'TimeStamp']

# لیست برای نگهداری دیتافریم‌های جداگانه
dfs = []

# پردازش هر AssetID به‌صورت جداگانه
for aid in asset_ids:
    df_aid = df[df['AssetID'] == aid][columns].copy()
    df_aid.columns = [f"{col}_{aid}" for col in columns]
    dfs.append(df_aid.reset_index(drop=True))

# ترکیب همه دیتافریم‌ها در محور افقی
df_final = pd.concat(dfs, axis=1)

# ذخیره فایل نهایی
df_final.to_csv('flattened_by_assetid.csv', index=False)

print("✅ فایل نهایی با ستون‌های جداگانه برای هر AssetID ذخیره شد: flattened_by_assetid.csv")


✅ فایل نهایی با ستون‌های جداگانه برای هر AssetID ذخیره شد: flattened_by_assetid.csv


In [44]:
# import pandas as pd

# # بارگذاری فایل خروجی قبلی
# df = pd.read_csv('flattened_by_assetid.csv')

# # لیست AssetIDها
# asset_ids = [9362, 9363, 9364, 9365, 9366, 9367, 9371, 9372]

# # لیست برای نگهداری ردیف‌های نهایی
# final_rows = []

# # تعداد ردیف‌ها
# n = len(df)

# # پیمایش ردیف‌ها
# for i in range(n):
#     row = df.iloc[i]
#     base_values = {}
#     valid = True

#     for aid in asset_ids:
#         base_ts = row[f'TimeStamp_{aid}']
#         if pd.isna(base_ts):
#             valid = False
#             break

#         # تبدیل به عدد برای محاسبه اختلاف
#         try:
#             base_ts = float(base_ts)
#         except:
#             valid = False
#             break

#         # جستجوی نزدیک‌ترین TimeStamp در 5 ردیف بعدی
#         min_diff = float('inf')
#         closest_value = None

#         for j in range(i+1, min(i+6, n)):
#             next_ts = df.iloc[j][f'TimeStamp_{aid}']
#             if pd.isna(next_ts):
#                 continue
#             try:
#                 next_ts = float(next_ts)
#             except:
#                 continue

#             diff = abs(next_ts - base_ts)
#             if diff < 15000 and diff < min_diff:
#                 min_diff = diff
#                 closest_value = df.iloc[j][f'Value_{aid}']

#         if closest_value is not None:
#             base_values[f'Value_{aid}'] = closest_value
#         else:
#             valid = False
#             break

#     if valid:
#         # ذخیره TimeStamp عددی
#         base_values['BaseTimeStamp'] = base_ts
#         # تبدیل به datetime و ذخیره در ستون جداگانه
#         base_values['BaseDateTime'] = pd.to_datetime(base_ts, unit='s')
#         final_rows.append(base_values)

# # ساخت دیتافریم نهایی
# df_final = pd.DataFrame(final_rows)

# # ذخیره فایل نهایی
# df_final.to_csv('matched_values_with_datetime.csv', index=False)

# print("✅ فایل نهایی با ستون datetime قابل استفاده در تایم‌سریز ذخیره شد: matched_values_with_datetime.csv")


In [45]:
# بارگذاری فایل خروجی قبلی
df = pd.read_csv('flattened_by_assetid.csv')

# لیست AssetIDها
asset_ids = [9362, 9363, 9364, 9365, 9366, 9367, 9371, 9372]

# لیست برای نگهداری ردیف‌های نهایی
final_rows = []

# تعداد ردیف‌ها
n = len(df)

# پیمایش ردیف‌ها
for i in range(n):
    row = df.iloc[i]
    base_values = {}
    valid = True

    for aid in asset_ids:
        base_ts = row[f'TimeStamp_{aid}']
        if pd.isna(base_ts):
            valid = False
            break

        # تبدیل به عدد برای محاسبه اختلاف
        try:
            base_ts = float(base_ts)
        except:
            valid = False
            break

        # جستجوی نزدیک‌ترین TimeStamp در 5 ردیف بعدی
        min_diff = float('inf')
        closest_value = None
        matched_ts = None

        for j in range(i+1, min(i+6, n)):
            next_ts = df.iloc[j][f'TimeStamp_{aid}']
            if pd.isna(next_ts):
                continue
            try:
                next_ts = float(next_ts)
            except:
                continue

            diff = abs(next_ts - base_ts)
            if diff < 3600 and diff < min_diff:
                min_diff = diff
                closest_value = df.iloc[j][f'Value_{aid}']
                matched_ts = next_ts

        if closest_value is not None and matched_ts is not None:
            base_values[f'Value_{aid}'] = closest_value
            base_values[f'MatchedTimeStamp_{aid}'] = pd.to_datetime(matched_ts, unit='s')
        else:
            valid = False
            break

    if valid:
        base_values['BaseTimeStamp'] = base_ts
        base_values['BaseDateTime'] = pd.to_datetime(base_ts, unit='s')
        final_rows.append(base_values)

# ساخت دیتافریم نهایی
df_final = pd.DataFrame(final_rows)

# ذخیره فایل نهایی
df_final.to_csv('matched_values_with_timestamps.csv', index=False)

print("✅ فایل نهایی با مقادیر Value و TimeStampهای تطبیق‌یافته ذخیره شد: matched_values_with_timestamps.csv")


✅ فایل نهایی با مقادیر Value و TimeStampهای تطبیق‌یافته ذخیره شد: matched_values_with_timestamps.csv
